## Semantic Search - Vectorize

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./source/idioms.csv')

In [3]:
df.head()

,Description,Meaning,Source,Type
0,가갸 뒤 자〔뒷다리〕도 모른다,"글자를 모르는 사람을 비유적으로 이르는 말.,사리에 어두운 사람을 놀림조로 이르는 말.",고려대 한국어대사전,속담
1,가까운 남이 먼 일가보다 낫다,이웃끼리 서로 친하게 지내다 보면 먼 곳에 있는 일가보다 더 친하게 되어 서로 도우...,표준국어대사전,속담
2,가까운 데 집은 깎이고 먼 데 절은 비친다,가까운 데 있는 절이나 집은 자잘한 흠도 잘 드러나서 좋지 않아 보이고 먼 곳에 윤...,표준국어대사전,속담
3,가까운 무당보다 먼 데 무당이 영하다,흔히 사람은 자신이 잘 알고 가까이 있는 것보다는 잘 모르고 멀리 있는 것을 더 좋...,표준국어대사전,속담
4,가까운 집 며느리일수록 흉이 많다,늘 가까이 있고 잘 아는 사이일수록 상대편의 결점이 눈에 더 많이 띈다는 말.,표준국어대사전,속담


In [5]:
# get unique source from dataframe
source = df['Type'].unique()
source

array(['속담', '관용구'], dtype=object)

In [6]:
len(df[df['Type'] == '속담'])

4480

In [11]:
df_proverb = df[df['Type'] == '속담'].copy()
df_proverb.head()

,Description,Meaning,Source,Type
0,가갸 뒤 자〔뒷다리〕도 모른다,"글자를 모르는 사람을 비유적으로 이르는 말.,사리에 어두운 사람을 놀림조로 이르는 말.",고려대 한국어대사전,속담
1,가까운 남이 먼 일가보다 낫다,이웃끼리 서로 친하게 지내다 보면 먼 곳에 있는 일가보다 더 친하게 되어 서로 도우...,표준국어대사전,속담
2,가까운 데 집은 깎이고 먼 데 절은 비친다,가까운 데 있는 절이나 집은 자잘한 흠도 잘 드러나서 좋지 않아 보이고 먼 곳에 윤...,표준국어대사전,속담
3,가까운 무당보다 먼 데 무당이 영하다,흔히 사람은 자신이 잘 알고 가까이 있는 것보다는 잘 모르고 멀리 있는 것을 더 좋...,표준국어대사전,속담
4,가까운 집 며느리일수록 흉이 많다,늘 가까이 있고 잘 아는 사이일수록 상대편의 결점이 눈에 더 많이 띈다는 말.,표준국어대사전,속담


In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from openai import AzureOpenAI

import os

client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_KEY"),  
  api_version = "2025-01-01-preview",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [9]:
def generate_embeddings(text, model="text-embedding-3-large"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# batch embeddings
def generate_embeddings_batch(texts, model="text-embedding-3-large"): # model = "deployment_name"
    response = client.embeddings.create(input=texts, model=model)
    return [vector.embedding for vector in response.data]

from tqdm import tqdm

# Parameters
batch_size = 20  # Number of rows to process in a batch, adjust as needed

# Function to get embeddings in batches
def get_embeddings_in_batches(df, batch_size, model_name="text-embedding-3-large"):
    embeddings = []
    for i in tqdm(range(0, len(df), batch_size)):
        # Prepare the batch
        batch_content = df['meaning'].iloc[i:i + batch_size].tolist()

        # Request embeddings from OpenAI in batch
        response = client.embeddings.create(
            input=batch_content,
            model=model_name
        )
        
        # Extract embeddings
        batch_embeddings = [item.embedding for item in response.data]
        embeddings.extend(batch_embeddings)
    
    return embeddings


In [14]:
text = df_proverb['Meaning'].iloc[0]
embeddings = generate_embeddings(text)
embeddings

[0.017118478193879128,
 -0.024258537217974663,
 -0.01571821980178356,
 0.008679081685841084,
 0.00796633679419756,
 0.011649901047348976,
 -0.02121833525598049,
 -0.02790425531566143,
 -0.012129268608987331,
 -0.062469203025102615,
 -0.012223880738019943,
 -3.062573159695603e-05,
 0.02017129585146904,
 0.010224412195384502,
 -0.035599373281002045,
 0.03850080817937851,
 0.01648773066699505,
 0.008470934815704823,
 -0.01824120804667473,
 -0.009335058741271496,
 0.005582112818956375,
 -0.033454831689596176,
 -0.020764198154211044,
 -0.004137701354920864,
 0.030477704480290413,
 0.00872954074293375,
 0.03244563564658165,
 0.03254655376076698,
 -0.04821431636810303,
 -0.025721870362758636,
 0.009341365657746792,
 -0.004412076435983181,
 0.05187264829874039,
 0.006969756446778774,
 -0.027399657294154167,
 -0.02694552019238472,
 0.003044932149350643,
 -0.001739285420626402,
 0.0056073423475027084,
 -0.024258537217974663,
 -0.043874774128198624,
 -0.03247086703777313,
 -0.037668220698833466,


In [13]:
texts = df_proverb['Meaning'].iloc[0:2].to_list()
embeddings_list = generate_embeddings_batch(texts)
len(embeddings_list)

2

In [ ]:
df_proverb['Embeddings3'] = get_embeddings_in_batches(df, batch_size)
df_proverb.head()

In [ ]:
df_proverb['Embeddings2'] = get_embeddings_in_batches(df, batch_size, model_name="text-embedding-ada-002")
df_proverb.head()

In [ ]:
df_proverb.to_pickle('kor_proverbs_embeddings.pkl')